In [ ]:
from itertools import chain
from pathlib import PosixPath

import pandas as pd

pd.set_option("display.max_colwidth", None)

In [ ]:
RAW_DATA = PosixPath("~/CTDS/projects/midrc/processed-s3/acrimage").expanduser()

# for RemoveHeads files, they have the same structure and needs to remove imaging_study
# which is `submitter_id` in the format of
# <case>_<study_id>
remove_heads_files = RAW_DATA.glob("**/RemoveHeads*.txt")
remove_heads_studies = map(lambda v: pd.read_csv(v, sep="\t"), remove_heads_files)
remove_heads_studies = map(lambda v: v["submitter_id"] \
        .str.split("_", expand=True) \
        .rename(columns={0: "case_id", 1: "study_id"})[["study_id"]],
    remove_heads_studies)
remove_heads_studies = pd.concat(remove_heads_studies).reset_index(drop=True)

# same thing for different format of deletion files
# there are two different formats: one for imaging_study and one for images :facepalm:
# this needs some column renaming
rename_columns = {
    "*type": "type",
    "*submitter_id": "submitter_id",
    "study_uid": "study_id",
}

deletion_imaging_study_files = RAW_DATA.glob("**/deletion_*.tsv")
deletion_imaging_study_studies = map(lambda v: pd.read_csv(v, sep="\t") \
        .rename(columns=rename_columns),
    deletion_imaging_study_files)
deletion_imaging_study_studies = map(lambda v: v[["study_id"]], deletion_imaging_study_studies)
deletion_imaging_study_studies = pd.concat(deletion_imaging_study_studies).reset_index(drop=True)

studies_to_delete = pd.concat([remove_heads_studies, deletion_imaging_study_studies]).reset_index(drop=True)

In [ ]:
removal_files = list(chain(
    # RAW_DATA.glob("**/deletion_*.tsv"),
    RAW_DATA.glob("**/RemoveHeads*.txt"),
))

In [ ]:
for r in removal_files:
    print(r)

In [ ]:
removal = pd.concat(map(lambda v: pd.read_csv(v, sep="\t"), removal_files))[[]]

In [ ]:
indexed_file = "/Users/andrew/CTDS/projects/midrc/midrc_indexing_scripts/old/indexed/indexing/archive/index_rsna_2021_08/p_acr_2021_06-07_indexing-output-manifest_filenames_correct_filenames.tsv"
deletion_file = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/07/deletion_imaging_study_ACR_202107.tsv"
indexed = pd.read_csv(indexed_file, sep="\t")
deletion = pd.read_csv(deletion_file, sep="\t")
sub_indexed = indexed[["guid", "file_name"]].copy()
sub_indexed.loc[:, "file_name"] = sub_indexed.loc[:, "file_name"].apply(lambda v: v.split("/")[-1])
sub_deletion = pd.DataFrame(data=deletion["storage_urls"].apply(lambda v: v.strip().split("/")[-1]))
sub_deletion.loc[0].values
sub_indexed.merge(sub_deletion, left_on="file_name", right_on="storage_urls").to_csv("/Users/andrew/Downloads/07.csv", index=False)

In [ ]:
indexed_file = "/Users/andrew/CTDS/projects/midrc/midrc_indexing_scripts/old/indexed/indexing/archive/08-0827/indexed_08.tsv"
deletion_file = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/08/deletion_imaging_study_ACR_202108.tsv"
indexed = pd.read_csv(indexed_file, sep="\t")
deletion = pd.read_csv(deletion_file, sep="\t")
sub_indexed = indexed[["guid", "file_name"]].copy()
sub_indexed.loc[:, "file_name"] = sub_indexed.loc[:, "file_name"].apply(lambda v: v.split("/")[-1])
sub_deletion = pd.DataFrame(data=deletion["storage_urls"].apply(lambda v: v.strip().split("/")[-1]))
sub_deletion.loc[0].values
sub_indexed.merge(sub_deletion, left_on="file_name", right_on="storage_urls").to_csv("/Users/andrew/Downloads/08.csv", index=False)

In [ ]:
indexed_file = "/Users/andrew/CTDS/projects/midrc/midrc_indexing_scripts/old/indexed/indexing/acr/indexed_image_20210903.txt"
deletion_file = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/09/RemoveHeadsFrom20210903.txt"
indexed = pd.read_csv(indexed_file, sep="\t")
deletion = pd.read_csv(deletion_file, sep="\t")
sub_indexed = indexed[["guid", "file_name"]].copy()
sub_indexed.loc[:, "file_name"] = sub_indexed.loc[:, "file_name"].apply(lambda v: v.split("/")[0])
sub_deletion = pd.DataFrame(data=deletion["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_indexed.merge(sub_deletion, left_on="file_name", right_on="study_uid")
sub_indexed.merge(sub_deletion, left_on="file_name", right_on="study_uid").to_csv("data/09.csv", index=False)

In [ ]:
# indexed_file = "/Users/andrew/CTDS/projects/midrc/indexing-data/old/indexed/01_31_22_23:32:42_bqcnja.log"
deletion_file = "/Users/andrewprokhorenkov/CTDS/projects/midrc/s3-data/raw/acrimage/2021/10/batch6/RemoveHeadsFrom20210914.txt"
deletion_file_2 = "/Users/andrewprokhorenkov/CTDS/projects/midrc/s3-data/raw/acrimage/2021/10/deletion_imaging_study_ACR_202110.tsv"
# indexed = pd.read_csv(indexed_file, sep="\t")
deletion = pd.read_csv(deletion_file, sep="\t")
deletion_2 = pd.read_csv(deletion_file_2, sep="\t")
deletion

sub_indexed = indexed[["guid", "file_name"]].copy()
sub_indexed.loc[:, "file_name"] = sub_indexed.loc[:, "file_name"].apply(lambda v: v.split("/")[1])
sub_deletion = pd.DataFrame(data=deletion["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_deletion_2 = pd.DataFrame(data=deletion_2["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_deletion_total = pd.concat([sub_deletion, sub_deletion_2])
sub_indexed.merge(sub_deletion_total, left_on="file_name", right_on="study_uid")
sub_indexed.merge(sub_deletion_total, left_on="file_name", right_on="study_uid").to_csv("data/10_batch6_seq.csv", index=False)

In [ ]:
indexed_file = "/Users/andrew/CTDS/projects/midrc/midrc_indexing_scripts/old/indexed/02_01_22_15:19:22_fjnhvt.log"
indexed_file_2 = "/Users/andrew/CTDS/projects/midrc/midrc_indexing_scripts/old/indexed/02_02_22_07:09:30_ethlyx.log"
deletion_file = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/10/batch6/RemoveHeadsFrom20210914.txt"
deletion_file_2 = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/10/deletion_imaging_study_ACR_202110.tsv"
indexed = pd.read_csv(indexed_file, sep="\t")
indexed_2 = pd.read_csv(indexed_file_2, sep="\t")
deletion = pd.read_csv(deletion_file, sep="\t")
deletion_2 = pd.read_csv(deletion_file_2, sep="\t")
sub_indexed = indexed[["guid", "file_name"]].copy()
sub_indexed.loc[:, "file_name"] = sub_indexed.loc[:, "file_name"].apply(lambda v: v.split("/")[1])
sub_indexed_2 = indexed_2[["guid", "file_name"]].copy()
sub_indexed_2.loc[:, "file_name"] = sub_indexed_2.loc[:, "file_name"].apply(lambda v: v.split("/")[1])
sub_deletion = pd.DataFrame(data=deletion["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_deletion_2 = pd.DataFrame(data=deletion_2["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_indexed_total = pd.concat([sub_indexed, sub_indexed_2])
sub_deletion_total = pd.concat([sub_deletion, sub_deletion_2])
sub_indexed_total
sub_indexed_total.merge(sub_deletion_total, left_on="file_name", right_on="study_uid")
sub_indexed_total.merge(sub_deletion_total, left_on="file_name", right_on="study_uid").to_csv("data/10_batch6.csv", index=False)

In [ ]:
indexed_file = "/Users/andrew/CTDS/projects/midrc/midrc_indexing_scripts/old/indexed/01_04_22_00:02:21_sfqnve.log"
deletion_file = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/10/batch7/RemoveHeadsFrom20210923.txt"
deletion_file_2 = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/10/deletion_imaging_study_ACR_202110.tsv"
indexed = pd.read_csv(indexed_file, sep="\t")
deletion = pd.read_csv(deletion_file, sep="\t")
deletion_2 = pd.read_csv(deletion_file_2, sep="\t")
sub_indexed = indexed[["guid", "file_name"]].copy()
sub_indexed.loc[:, "file_name"] = sub_indexed.loc[:, "file_name"].apply(lambda v: v.split("/")[1])
sub_deletion = pd.DataFrame(data=deletion["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_deletion_2 = pd.DataFrame(data=deletion_2["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_deletion_total = pd.concat([sub_deletion, sub_deletion_2])
sub_indexed.merge(sub_deletion_total, left_on="file_name", right_on="study_uid")
sub_indexed.merge(sub_deletion_total, left_on="file_name", right_on="study_uid").to_csv("data/10_batch7_seq.csv", index=False)

In [ ]:
indexed_file = "/Users/andrew/CTDS/projects/midrc/midrc_indexing_scripts/old/indexed/01_04_22_11:45:36_drtraw.log"
indexed_file_2 = "/Users/andrew/CTDS/projects/midrc/midrc_indexing_scripts/old/indexed/01_05_22_10:11:21_bprxsq.log"
deletion_file = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/10/batch7/RemoveHeadsFrom20210923.txt"
deletion_file_2 = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/10/deletion_imaging_study_ACR_202110.tsv"
indexed = pd.read_csv(indexed_file, sep="\t")
indexed_2 = pd.read_csv(indexed_file_2, sep="\t")
deletion = pd.read_csv(deletion_file, sep="\t")
deletion_2 = pd.read_csv(deletion_file_2, sep="\t")
sub_indexed = indexed[["guid", "file_name"]].copy()
sub_indexed.loc[:, "file_name"] = sub_indexed.loc[:, "file_name"].apply(lambda v: v.split("/")[1])
sub_indexed_2 = indexed_2[["guid", "file_name"]].copy()
sub_indexed_2.loc[:, "file_name"] = sub_indexed_2.loc[:, "file_name"].apply(lambda v: v.split("/")[1])
sub_deletion = pd.DataFrame(data=deletion["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_deletion_2 = pd.DataFrame(data=deletion_2["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_indexed_total = pd.concat([sub_indexed, sub_indexed_2])
sub_deletion_total = pd.concat([sub_deletion, sub_deletion_2])
sub_indexed_total.merge(sub_deletion_total, left_on="file_name", right_on="study_uid")
sub_indexed_total.merge(sub_deletion_total, left_on="file_name", right_on="study_uid").to_csv("data/10_batch7.csv", index=False)

In [ ]:
indexed_file = "/Users/andrew/CTDS/projects/midrc/midrc_indexing_scripts/old/indexed/indexing/acr/indexed_image_20210903.txt"
deletion_file = "/Users/andrew/CTDS/projects/midrc/s3-data/raw/acrimage/2021/11/RemoveHeadsFrom20211004.txt"
indexed = pd.read_csv(indexed_file, sep="\t")
deletion = pd.read_csv(deletion_file, sep="\t")
sub_indexed = indexed[["guid", "file_name"]].copy()
sub_indexed.loc[:, "file_name"] = sub_indexed.loc[:, "file_name"].apply(lambda v: v.split("/")[0])
sub_deletion = pd.DataFrame(data=deletion["study_uid"].apply(lambda v: v.strip().split("/")[0]))
sub_indexed.merge(sub_deletion, left_on="file_name", right_on="study_uid")
sub_indexed.merge(sub_deletion, left_on="file_name", right_on="study_uid").to_csv("data/09.csv", index=False)